In [3]:
   import pandas as pd
   import numpy as np

   # Loading the data
   df = pd.read_csv('final_pose_features_dataset.csv')

   # Ensure proper sorting
   df = df.sort_values(by=['video_name', 'frame_id'])

   # Identify feature columns (excluding ID and label)
   feature_cols = [col for col in df.columns if col not in ['video_name', 'frame_id', 'label_x', 'label_y']]

   # Group by video and stack sequences
   X = []
   y = []

   for video_id, group in df.groupby('video_name'):
       group = group.sort_values('frame_id')
       features = group[feature_cols].values
       label = group['label_x'].iloc[0]  # Assuming label per video

       X.append(features)
       y.append(label)

   # Padding sequences to same length
   from tensorflow.keras.preprocessing.sequence import pad_sequences

   X = pad_sequences(X, padding='post', dtype='float32')  # shape -> (num_videos, max_frames, num_features)
   y = np.array(y)

   # Encoding the labels
   from sklearn.preprocessing import LabelEncoder

   le = LabelEncoder()
   y_encoded = le.fit_transform(y)

   num_classes = len(le.classes_)
   print("Number of classes:", num_classes)

   from sklearn.model_selection import train_test_split

   # Split data into training and testing sets (80% train, 20% test)
   X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

   # Further split the test set into validation and test sets (50% validation, 50% test)
   X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)


Number of classes: 8


In [4]:
import tensorflow as tf
import gcsfs

# Specify the GCS bucket and model file path
BUCKET_NAME = 'exercise-recognition-dataset'  # Replace with your bucket name
MODEL_PATH = 'model/tcn_pose_model.keras'  # Replace with the path to your .keras file

# Construct the full GCS path
gcs_model_path = f'gs://{BUCKET_NAME}/{MODEL_PATH}'

# Load the model
model = tf.keras.models.load_model(gcs_model_path)

model.compile(optimizer='adam',  # or any other optimizer
              loss='sparse_categorical_crossentropy',  # or your loss function
              metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'tcn_2' (of type TCN) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [5]:
epochs = 50  # Choose the number of epochs
batch_size = 20 # Choose the batch size

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'conv1D_0' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'matching_conv1D' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'residual_block_0' (of type ResidualBlock) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.8377 - loss: 0.4381 - val_accuracy: 0.4646 - val_loss: 3.1889
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8959 - loss: 0.2718 - val_accuracy: 0.4242 - val_loss: 3.4333
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8809 - loss: 0.3418 - val_accuracy: 0.4242 - val_loss: 3.2227
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8713 - loss: 0.3332 - val_accuracy: 0.4242 - val_loss: 4.0464
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9064 - loss: 0.2616 - val_accuracy: 0.4646 - val_loss: 3.9443
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8812 - loss: 0.3087 - val_accuracy: 0.4242 - val_loss: 3.3694
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8802 - loss: 0.3363 - val_accuracy: 0.4444 - val_loss: 3.4133
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.8720 - loss: 0.3472 - val_accuracy: 0.4545 - val_loss: 3.

In [6]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Test Loss: 6.0527
Test Accuracy: 0.4600


In [7]:
# Save back to GCS
FINE_TUNED_MODEL_PATH = 'model/fine_tuned_model_1.keras'  # Define the GCS path
gcs_fine_tuned_model_path = f'gs://{BUCKET_NAME}/{FINE_TUNED_MODEL_PATH}'
model.save(gcs_fine_tuned_model_path)

# Saving the model
model.save('fine_tuned_model_1.keras')

Fine-Tune 2

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Reduce learning rate
model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
epochs = 50  # Choose the number of epochs
batch_size = 20 # Choose the batch size

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.9432 - loss: 0.1632 - val_accuracy: 0.4545 - val_loss: 3.6085
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9783 - loss: 0.0563 - val_accuracy: 0.4747 - val_loss: 3.6483
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9741 - loss: 0.0868 - val_accuracy: 0.4747 - val_loss: 3.5211
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9875 - loss: 0.0462 - val_accuracy: 0.4747 - val_loss: 3.5449
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9783 - loss: 0.0605 - val_accuracy: 0.4747 - val_loss: 3.5814
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9922 - loss: 0.0316 - val_accuracy: 0.4747 - val_loss: 3.6788
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9853 - loss: 0.0395 - val_accuracy: 0.4646 - val_loss: 3.8025
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9906 - loss: 0.0386 - val_accuracy: 0.4646 - v

In [9]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Test Loss: 8.6931
Test Accuracy: 0.5000


In [10]:
# Save back to GCS
FINE_TUNED_MODEL_PATH = 'model/fine_tuned_model_2.keras'  # Define the GCS path
gcs_fine_tuned_model_path = f'gs://{BUCKET_NAME}/{FINE_TUNED_MODEL_PATH}'
model.save(gcs_fine_tuned_model_path)

# Saving the model
model.save('fine_tuned_model_2.keras')